In [4]:
cd ../python/

/Users/ashkanaleali/w/Trader/python


In [12]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime

from arctic import Arctic
import quandl

from utilities import *

In [14]:
from zipline.api import symbol

In [2]:
%load_ext zipline

# Dual Moving Average Strategy

The idea is that when the 20 moving average, which reacts faster, moves above the 50 moving average, it means the price might be trending up, and we may want to invest. Conversely, if the 20 moving average falls below the 50 moving average, this signals maybe that the price is trending down, and that we might want to either sell or investment or even short sell the company.

In [28]:
%%zipline --start 2017-1-1 --end 2018-6-1 -o portfolio/dual_moving_average.pickle

from zipline.api import *

def initialize(context):
    context.security = symbol('AAPL')
    
def handle_data(context, data):
    MA1 = data.history(context.security, 'price', bar_count=50, frequency="1d").mean()
    MA2 = data.history(context.security, 'price', bar_count=200, frequency="1d").mean()
    
    current_price = data[context.security].price
    current_positions = context.portfolio.positions[symbol('AAPL')].amount
    cash = context.portfolio.cash
    
    if MA1 > MA2 and current_positions == 0:
        number_of_shares = int(cash / current_price)
        order(context.security, number_of_shares)
#         log.info("Buying shares")
        
    if MA1 < MA2 and current_positions > 0:
        order_target(context.security, 0)
#         log.info("Selling shares")
    
    record(MA1=MA1, MA2=MA2, price=current_price)

ValueError: cannot convert float NaN to integer